In [30]:
import pandas as pd
import re

In [57]:
filename = 'testing.csv'
df = pd.read_csv(filename)
df.drop(df.columns[0], axis=1, inplace=True)
df.drop_duplicates('link', inplace=True)
df.set_index('link', inplace=True)
df.head()

,date,post,title
link,,,
/r/mechmarket/comments/n2c4ws/usmi_h_e65_ewhite_w_lots_of_extras_58x_lf_mode/,1.619847e+09,[Timestamp](https://imgur.com/a/QZ3udpf)\n\n**...,"[US-MI] [H] E6.5 E-white /w lots of extras, 58..."
/r/mechmarket/comments/n29bmv/uswa_h_sealed_gmk_noel_sealed_gmmk_pro_black_w_pc/,1.619837e+09,[Timestamps](https://imgur.com/a/zudvbGx)\n\nH...,"[US-WA] [H] Sealed GMK Noel, Sealed GMMK Pro (..."
/r/mechmarket/comments/n292vu/usca_h_gmk_firefly_w_crp_tulip_kits_paypal/,1.619836e+09,[ts](https://imgur.com/a/GazrWHL)\n\nJust star...,[US-CA] [H] GMK Firefly [W] CRP Tulip Kits / P...
/r/mechmarket/comments/n28qlw/usca_h_kbd19x_gray_built_wzealios_v2_65g_lubed/,1.619834e+09,timestamp - [https://imgur.com/a/Y8sD8cc](http...,[US-CA] [H] KBD19x Gray Built w/Zealios v2 65g...
/r/mechmarket/comments/n27eii/usca_h_mode80_1st_edition_dark_inett_squarex_60/,1.619830e+09,[\[TIMESTAMP\]](https://imgur.com/a/aNYwOCO)\n...,"[US-CA] [H] Mode80 1st Edition (Dark), iNETT S..."


In [58]:
df['post_lower'] = df['post'].str.lower()
df['title_lower'] = df['title'].str.lower()
only_gmk = df[df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')].copy()
not_gmk = df[(~(df['post_lower'].str.contains('gmk') | df['title_lower'].str.contains('gmk')))].copy()

only_gmk.shape, df.shape

((774, 5), (777, 5))

In [59]:
not_gmk.head()

,date,post,title,post_lower,title_lower
link,,,,,
/r/mechmarket/comments/myvp2h/eult_h_modded_anne_pro_2_wmint_holy_pandas_mauve/,1.619437e+09,[TIMESTAMP](https://imgur.com/a/D4ORCTJ)\n\n[A...,[EU-LT] [H] Modded Anne Pro 2 W/Mint Holy Pand...,[timestamp](https://imgur.com/a/d4orctj)\n\n[a...,[eu-lt] [h] modded anne pro 2 w/mint holy pand...
/r/mechmarket/comments/mxqwzs/caon_h_rama_u80a_moon_extra_hs_pcb_w_paypal/,1.619292e+09,# Rama U80-A MOON - SOLD FOR ASKING $\n\n[time...,[CA-ON] [H] Rama U80-A MOON + Extra HS PCB [W]...,# rama u80-a moon - sold for asking $\n\n[time...,[ca-on] [h] rama u80-a moon + extra hs pcb [w]...
/r/mechmarket/comments/mmvuos/caon_h_epbt_ivory_gateron_yellow_ks3_lf_giant_v3/,1.617900e+09,Outdated,"[CA-ON] [H] ePBT Ivory, Gateron Yellow KS-3 L&...",outdated,"[ca-on] [h] epbt ivory, gateron yellow ks-3 l&..."


In [75]:
from collections import defaultdict
all_prices = defaultdict(list)

money_regex = r'\$(\d+)|(\d+)\$'

sets = ['base', 'nov', 'alpha', 'accent', 'bar', 'light base', 'dark base', 'deskmat', 'rama', '40s', '40\'s', 'fourties']

sales_data = []

def match_product(df):
    for row in df.itertuples():
        s = row.post_lower.split("\n")
        #product = row.product_name.lower()
        #print(row[0][1])
        for l in s:
            low = l.lower()
            if "gmk " in low and ('~~' in low or 'sold' in low):
                a = low.split("gmk ")[1]
                product_name = "gmk " + re.split(r"[^\w\+]", a)[0]
                
                matches = re.split(money_regex, low)
                temp_data = {}
                if len(matches) > 1:
                    print(f"FULL LINE: {low}")
                    for i in range(0, len(matches)-1, 3):
                        curr_price = int(matches[i+1] if matches[i+1] else matches[i+2])
                        curr_str = matches[i]
                        kits = []
                        removeBase = False
                        empty=True
                        for x in sets:
                            if x in curr_str:
                                empty=False
                                if x == 'nov':
                                    kits.append('novelties')
                                elif x == 'light base' or x == 'dark base':
                                    kits.insert(0, f'{x}')
                                    removeBase = True
                                elif x == 'bar':
                                    kits.append('spacebars')
                                elif x in ['40s', '40\'s', 'fourties']:
                                    kits.append('40s')
                                else:
                                    kits.append(x)
                        products = product_name
                        if i == 0 and not kits:
                            kits = ['base']
                        for x in kits:
                            if removeBase and x == 'base':
                                continue
                            products += f', {x}'
                        
                        if i == 0:
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        
                        if kits and i > 0:
                            print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                            #all_prices[temp_data['products']].append(temp_data['price'])
                            sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])
                            if temp_data['products'] == 'gmk dark|base, base, novelties':
                                print(row)
                            temp_data["products"] = products
                            temp_data["str"] = curr_str
                            temp_data["price"] = curr_price
                        else:
                            temp_data["price"] = min(temp_data["price"], curr_price)
                        
                    print(f"PRODUCT: {temp_data['products']} CURRENT STRING: {temp_data['str']} PRICE: {temp_data['price']}")
                    #all_prices[temp_data['products']].append(temp_data['price'])
                    sales_data.append([row[0], temp_data['products'], temp_data['price'], row.date])

for i, new_df in only_gmk.groupby(level=0):
    match_product(new_df)
    
#print(sales_data)

and numpad kits, rama artisan and matching coiled aviator cable. $450 obo for the entire set, will not part out at the moment.~~ sold for $420 without cable
PRODUCT: gmk cafe, alpha, spacebars, rama CURRENT STRING: ~~got some new some new stuff coming and need to make space so selling my gmk cafe set. the set is lightly used with some shine on the alpha keys. comes in the original box along with the spacebar and numpad kits, rama artisan and matching coiled aviator cable.  PRICE: 420
FULL LINE: **gmk oblivion assembly v2 / ortholinear / ergodox ~~($140)~~:** ***sold!***
PRODUCT: gmk oblivion, base CURRENT STRING: **gmk oblivion assembly v2 / ortholinear / ergodox ~~( PRICE: 140
FULL LINE: |**~~gmk godspeed armstrong base -~~** ~~used for about an hour/ no shine~~ **~~- $220 + shipping~~**|
PRODUCT: gmk godspeed, base CURRENT STRING: |**~~gmk godspeed armstrong base -~~** ~~used for about an hour/ no shine~~ **~~-  PRICE: 220
FULL LINE: |~~rama x gmk classic blue "x"~~|~~bnib, priority 

In [76]:
sales_df = pd.DataFrame(sales_data, columns=['link', 'product', 'price', 'date'])
sales_df['date'] = pd.to_datetime(sales_df['date'], unit = 's')
sales_df.to_csv('results_' + filename)